# Generation process of the Mock catalog

In this notebook we detail the modelling and calculation steps of the mock catalog generation described in Csörnyei & Dobos (2020)

In [ ]:
from astropy.cosmology import FlatLambdaCDM
from astropy.cosmology import z_at_value
from astropy import units as u
import scipy.interpolate
import scipy.signal
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
import os, os.path
from joblib import Parallel, delayed
import multiprocessing
import numpy

from LineSpec import generate_programs as gp

LIGHTSPEED = 299792.458  # km s-1
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.725)

As a first step the results of the PCA analysis have to be loaded, which are ordered into a python _dictionary_. The exact contents of the dictionary and the keys are detailed below.

In [ ]:
PCA_dict=numpy.load('./Generate_files/PCA_results_dict.npy').item()

This various PCA results are stored in the dictionary as follows:

  -   'Average Continuum' : The calculated average continuum of the model spectra. This is a _numpy array_, where the columns contain the wavelength and the flux values
  -  'Continuum Eigenvector' : This array contains the __righthandside__ eigenvectors (V) obtained by the Princial Component Analysis. These are also called eigenspectra. For our work (Csörnyei & Dobos (2020)), we only used the first five eigenspectra, since the individual spectra can be expressed with them precisely enough (up to about 97-99%). These eigenvectors are stored in a _numpy array_, in which rows contain the eigenspectra fluxes. The corresponding wavelength grid can be obtained by 'new_wl = numpy.linspace(3724, 6761, 5062)' as it was shown in the PCA notebook. 
  - 'Continuum Principal Components' : The principal components corresponding to the individual spectra. This is a _numpy array_, in which the rows contain the set of principal components for each spectra. By multiplying the Continuum eigenvector array from the left by this, we would get the model spectra back (with limited precision, which is controlled by the number of kept eigenspectra)
  - 'Average Emission Lines' : The calculated average emission line equivalent width logarithms. This is an _array_, in which one column contains the restframe line wavelengths, while in the other column contains the mentioned average logarithm line widths.
  - 'Emission Line Eigenvectors' : This _array_ contains the emission line eigenvectors in each row, similarly to the cace of the continuum datavectors.
  - 'Emission Line Principal Components' : The emission line principal components, stored the same way as the continuum principal components

In [ ]:
cont_avg=PCA_dict['Average Continuum']
cont_eig5=PCA_dict['Continuum Eigenvectors']
cont_PCs=PCA_dict['Continuum Principal Components']
line_avg=PCA_dict['Average Emission Lines']
line_eig=PCA_dict['Emission Line Eigenvectors']
line_PCs=PCA_dict['Emission Line Principal Components']

The distribution of the Continuum Principal Components vary in different absolute magnitude ranges: the brightest targets are most probably LRGs, which exhibit a passive spectrum without any emission lines, while the fainter, more active galaxies exhibit bluer colors, thus populate a different region the field of principal components. To model this, we split the PC coefficients into absolute magnitude ranges, then modelled the distributions in each with Gaussian Mixture models. The description of this model can be found on https://scikit-learn.org/stable/modules/mixture.html .

The magnitude ranges, which have contained numerous spectra were modelled with a maximal number of 15 individual Gaussian in the Mixture Model. The number of Gaussians scaled according to the number of spectra in the given magnitude range. To prevent overfitting, if the number of spectra in a range was below a given limit, then we just modelled it with 2 Gaussians. For details, see Csörnyei & Dobos (2020). This modelling step is not detailed in this notebook, we only load the results of the modelling specific for our dataset. 

The resulting Gaussian Mixture paramteres have been saved in a _python dictionary_, ordered by the brightness of the modelled ranges (from -24.0 to -16.5 absolute magnitude, with 0.5 width). These parameters can be directly used to create a random sample with the given distribution. This _dictionary_ is loaded below.

In [ ]:
gmm=numpy.load('./Generate_files/Continuum_gmm.npy').item()

The modelled spectra had a narrow wavelegth coverage, thus it could not be used to create random spectra on very high redshifts (otherwise we could not calculate synthetic magnitudes for the shortest wavelength filters). To lift this problem this we stiched the generated continuums by fitted model spectra on the infrared and on the ultraviolet ends. This had been done by expanding the spectra with stellar population models of different ages. The next script loads these models and apply logarithmic binning on them (which will be necessary for the fitting process).

In [ ]:
Z = (0.004, 0.008, 0.02, 0.05)      # The metallicities of the SSP-s
Z_txt = ('004', '008', '02', '05')
age = (0.005, 0.025, 0.1, 0.29, 0.64, 0.9, 1.4, 2.5, 5, 11)  # The ages
age_txt = ('5Myr', '25Myr', '100Myr', '290Myr', '640Myr', '900Myr', '1.4Gyr', '2.5Gyr', '5Gyr', '11Gyr')
temp_wl, temp_fl = gp.load_templates_1(Z, age)                # Here we load the templates
temp_log_wl, temp_log_fl = gp.spec_log_binning(temp_wl, temp_fl)  # The apply the logarithmic binning

The most important part of the generating process is to recreate the correct emission type distribution for the mock catalog. The main step during the generating process is to decide, whether the galaxy should be of strong emission line, or of non-emission (weak line, or passive) type. To model this weight maps for each absolute magnitude range have been created. We defined a lattice for each range in the Continuum Principal Component field in which we modelled the distributions in the previous step, then we counted the ratio of spectra that posessed all ten emission lines againts those, that did not in each of lattice bins (for reference, see Fig.3. in Csörnyei & Dobos (2020)). The former were classified as emission type, while the latter as non-emission. 

By this process, we obtained a set of grids, that contained float numbers from 0 to 1, which basically gives the probability of having strong emission lines on a galaxy, that in a given magnitude range falls into that lattice bin based on its continuum decomposition. In the simulation step, we will choose the appropriate weight map based on the absolute magnitude of the galaxy for each of the generated continuum spetra, with which we can decide whether the galaxy can exhibit strong emission lines by looking up the correct probability.

In [ ]:
weight_maps=numpy.load('./Generate_files/Weight_maps.npy').item()

In the upcoming steps we will make use of the weak correlation between the continuum spectra and the emission lines of galaxy spectra. According to this, if we see a particular continuum spectrum, then the set of lines that can appear on it will not be arbitrary, but they will follow a certain multi-dimensional distribution. To demonstrate this, we clustered the continuum principal components and then projected the obtained clusters to the field of the emission line principal components (see Fig.4. in Csörnyei & Dobos (2020)). As it can be seen on the figures presented there, the emission line principal components will not scatter to any arbitrary value, they are actually following a Gaussian-like distribution. The clustering was done with the __k-means__ clustering method, which gave the set of labels and the centers of the clusters as a result. The clusterwise divided principal components of continua and emission lines are stored in the _dictionary_ loaded below.

In [ ]:
Clustering = numpy.load('./Generate_files/Clustering.npy').item()
labels40 = Clustering['Labels']  # The labels and centers of the 40 clusters
centr40 = Clustering['Centers']

These Gaussian-like distributions were fitted on a similar way as the whole set of the continuum principal components, with Gaussian Mixture models. The number of Gaussians that were used in this step was 5, unless the number of galaxies in the given cluster was low, in which case 2 Gaussians were used.

In [ ]:
emi_gmm = numpy.load('./Generate_files/Emission_gmm.npy').item()

At this point we can start simulating spectra for our mock catalogue.

The first step of the catalog creation is to generate the redshifts and the absolute magnitudes. The redshifts should be drawn from an even distribution according to comoving volume, which is done via the use of the _comoving volume_ function from the _astropy_ package. The absolute magnitudes are drawn with the use of the Schechter-function. After the sets of absolute magnitudes and redshifts are drawn, we start pairing the values from the two sets randomly. For each pair the apparent magnitude is calculated (the luminosity distances are calculated with the help of _astropy_) with the formula of distance modulus and the pair is only kept, if the its apparent magnitude is lower than the magnitude limit of the chosen survey (now set to 17.7, for SDSS main sample). 

In [ ]:
z_samp = gp.gen_redshift(30000, 0.01, 0.4)                 # Set of redshifts
M_samp = gp.gen_M_sample(-24.5, -16.5, 10000, -21.53, -1.23)     # Set of absolute magnitudes

n = 2000                                              # Number of generated pairs
galaxies=numpy.zeros((n,2))
galaxies[:,0], galaxies[:,1], gmm_app = gp.gal_sample(n, cosmo, z_samp, M_samp ,17.7)    # Create pairs with the chosen limit magnitude

After the brightnesses and distances of the galaxies are calculated, a continuum spectrum must be generated and assigned to them, after which we have to decide whether we add emission lines or not. For the last step, some parameters have to be set before the simulation. The spectral binning and wavelength range must be supplied, as well as the theoretical waveleghts of the emission lines. The dispersion velocity has to be set as well (this will remain the same for the whole process).

In [ ]:
new_spec_wl = numpy.linspace(1000.0,11300.2,17168)     # The binning of spectrum

sigma = 200                                            # The dispersion velocity in km/s
emi_wl = numpy.array([ 3727.,  4342.,  4863.,  4960.,  5008.,  6550.,  6565.,  6585., 6718.,  6733.])  
                                                     # The wavelenghts of emission lines
    
spec_bin = new_spec_wl
D_lambd = emi_wl * (sigma / LIGHTSPEED)        # The widening of lines caused by the velocity dispersion
l_down = emi_wl - D_lambd                    # Lower limits of the emission lines
l_up = emi_wl + D_lambd                      # Upper limits
sig_lines = (l_up - emi_wl) / 5                # Considering a Gaussian profile, the spread of the line

Rather than finding the indices of the line boundaries in the total wavelength list by hand, this script finds them and stores both of them in one-one list

In [ ]:
res_fl, low_indices, up_indices = find_line_boundary_indices(spec_bin, emi_wl, sigma, l_up, l_down)

Since SDSS does not have wavelenght coverage in UV, nor in IR, these spectral parts have to be stiched by models which are appended to the simulated part. For this, we have to supply the wavelength ranges in which UV and IR spectra are needed.

Now we also need to set the parameters for the galaxies which are not in the strong emimssion line group. For these galaxies we will only add the H$\alpha$ emission line. We trat the equivalent widths of this line for those galaxies that does not exhibit complete emission profile (does not have all ten emission line) differently: we found that the strength of H$\alpha$ correlates directly with the combination of the continuum PCs, as seen on Fig.6. in Csörnyei & Dobos (2020). To make use of this correlation, we introduced a $d$ distance parameter in the filed of the first two continuum principal components, then used this to model equivalent width. This dependency could be modelled either with an exponential or with a linear curve (however, there is only a subtle difference between them). The equivalent width of the H$\alpha$ lines were fitted the exponential of the form 
$$f(d)=\alpha \cdot e^{(-\beta \cdot d+\gamma)}+\delta .$$
The parameters below are ordered according to the fitting formula seen before.

In [ ]:
new_wl_uv = numpy.linspace(1000,3724,4541)
new_wl_ir = numpy.linspace(6760.6,11300.2,7567)

hi_popt = numpy.array([7.31406729e+00,   3.07019774e-02,   3.33726852e+00,   -9.37943767e+01])

After loading the results of the models and setting all these parameters, the simulation can start. The generating step is done with the script below. The program will use multiprocess, the number of cpu-s used can be controlled via *num_cores*. The upper limit in inputs controls the number of created spectra. The spectra will be saved in separate files, in the directory specified in *final_directory*. The program goes through the galaxies array, which contains the redshifts and absolute magnitudes of the galaxies. In every loop the program generates continuum principal components from the correct modelled distribution with respect to the absoulte magnitude of the galaxy. These principal components will determine the next step: with the use of the previously introduced weight maps we determine the possibility that a given galaxy can be a strong emission line one. Two cases can occur here:
- ' Emission galaxy ' = If the galaxy we got is an emission galaxy, then we make use of the weak correlation present between the lines and the continuum. First, we find the nearest continuum cluster to our generated continuum principal components. After this cluster is found, we take the corresponding projected emission line cluster and with the use of the Gaussian Mixture models emission line principal components are drawn. These principal components exactly describe the emission lines that will appear on the continuum, thus the whole spectrum is created by just adding the continuum and the lines.
- 'Non-emission galaxy' = If the galaxy is found not to be an emission one, then a different approach is used. Instead of using the weak correlation between the continuum and the lines, and adding a total emission profile, we only add the H$\alpha$ line, for which we use the calculated exponential (or linear) model. We found that the distance from a hypothetical point on the continuum principal component field and an applied random scatter will give H$\alpha$ emission line widths that correspond well to reality. For non-emission galaxies this distance is calculated, then the above exponential model is applied, then a random error is added. The result will be an equivalent width, for which an appropriate line profile is calculated and the total spectrum is calculated.

As a final step, the resulting spectra are shifted to their redshift, and they are finally saved to the target dictionary into indiviual files.

For an easier overview on the whole process we refer the reader to Fig.8. in Csörnyei & Dobos (2020).

In [ ]:
final_spectra={}
num_cores = int(multiprocessing.cpu_count() // 5)
inputs = range(0, 20)
final_directory = './RunTemp/'

results = Parallel(n_jobs = num_cores)(delayed(processInput)(i, final_spectra,\
                galaxies, final_directory, hi_popt, cont_avg, cont_eig5, gmm, emi_gmm, new_spec_wl,\
                temp_log_wl, temp_log_fl, weight_maps, centr40, sig_lines, low_indices, up_indices, l_down, l_up,\
                line_eig, line_avg, emi_wl) for i in inputs)